# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from env import user, password, host
import warnings
warnings.filterwarnings('ignore')

# Aquire

In [2]:
url = f"mysql+pymysql://{user}:{password}@{host}/zillow"

query = '''
        SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips
        FROM properties_2017
        LEFT JOIN propertylandusetype USING(propertylandusetypeid)
        WHERE propertylandusedesc IN ('Single Family Residential', 'Inferred Single Family Residential')
        '''

df = pd.read_sql(query, url)
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [3]:
df.describe()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
count,2.152853e+06,2.152853e+06,2.144379e+06,2.152371e+06,2.143526e+06,2.148422e+06,2.152864e+06
mean,3.287195e+00,2.230687e+00,1.862855e+03,4.618961e+05,1.960950e+03,5.634864e+03,6.048377e+03
std,9.547568e-01,9.992805e-01,1.222125e+03,6.996759e+05,2.216220e+01,8.178909e+03,2.043329e+01
min,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.801000e+03,1.850000e+00,6.037000e+03
25%,3.000000e+00,2.000000e+00,1.257000e+03,1.881700e+05,1.949000e+03,2.534972e+03,6.037000e+03
50%,3.000000e+00,2.000000e+00,1.623000e+03,3.276710e+05,1.958000e+03,4.108945e+03,6.037000e+03
75%,4.000000e+00,3.000000e+00,2.208000e+03,5.345270e+05,1.976000e+03,6.414318e+03,6.059000e+03
max,2.500000e+01,3.200000e+01,9.525760e+05,9.842891e+07,2.016000e+03,1.337756e+06,6.111000e+03


In [4]:
df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152864 entries, 0 to 2152863
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2152853 non-null  float64
 1   bathroomcnt                   2152853 non-null  float64
 2   calculatedfinishedsquarefeet  2144379 non-null  float64
 3   taxvaluedollarcnt             2152371 non-null  float64
 4   yearbuilt                     2143526 non-null  float64
 5   taxamount                     2148422 non-null  float64
 6   fips                          2152864 non-null  float64
dtypes: float64(7)
memory usage: 115.0 MB


In [5]:
# Looks like fips is the only column without any null or missing values
# Lets check the dmg
df.isnull().sum()

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8485
taxvaluedollarcnt                493
yearbuilt                       9338
taxamount                       4442
fips                               0
dtype: int64

In [6]:
# These column names are too much
# Rename Columns

df = df.rename(columns = {
                          'bedroomcnt':'bedrooms', 
                          'bathroomcnt':'bathrooms', 
                          'calculatedfinishedsquarefeet':'sqr_feet',
                          'taxvaluedollarcnt':'tax_value', 
                          'yearbuilt':'year_built'})

In [10]:
df.head(3)

,bedrooms,bathrooms,sqr_feet,tax_value,year_built,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0


In [18]:
# Drop the null values so that our data is consistent
df = df.dropna()

In [19]:
# I also feel our  year built and fips should not be float dtypes
# The other columns are proper counts of stuff, but year built and fips apear to be more catagorical

df.year_built.value_counts()

1955.0    84329
1950.0    71606
1954.0    69069
1953.0    62270
1956.0    61858
          ...  
1855.0        1
1879.0        1
1833.0        1
1840.0        1
1874.0        1
Name: year_built, Length: 153, dtype: int64

In [20]:
df.fips.value_counts()

6037.0    1425207
6059.0     552057
6111.0     162971
Name: fips, dtype: int64

In [21]:
# Yes, changing year_built and fips top dtype(object)
df.year_built = df.year_built.astype(object)
df.fips = df.fips.astype(object)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152863
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   bedrooms    float64
 1   bathrooms   float64
 2   sqr_feet    float64
 3   tax_value   float64
 4   year_built  object 
 5   taxamount   float64
 6   fips        object 
dtypes: float64(5), object(2)
memory usage: 130.6+ MB
